In [1]:
import numpy as np
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from skimage.io import imshow
from keras import backend as K
from keras.constraints import Constraint
from keras.datasets import cifar10
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
num_classes=10
img_rows, img_cols = 32, 32

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
def inference_via_confidence(confidence_mtx1, confidence_mtx2, label_vec1, label_vec2):

    #----------------First step: obtain confidence lists for both training dataset and test dataset--------------
    confidence1 = []
    confidence2 = []
    acc1 = 0
    acc2 = 0
    for num in range(confidence_mtx1.shape[0]):
        confidence1.append(confidence_mtx1[num,label_vec1[num]])
        if np.argmax(confidence_mtx1[num,:]) == np.argmax(label_vec1[num]):
            acc1 += 1

    for num in range(confidence_mtx2.shape[0]):
        confidence2.append(confidence_mtx2[num,label_vec2[num]])
        if np.argmax(confidence_mtx2[num,:]) == np.argmax(label_vec2[num]):
            acc2 += 1
    confidence1 = np.array(confidence1)
    confidence2 = np.array(confidence2)

    print('model accuracy for training and test-', (acc1/confidence_mtx1.shape[0], acc2/confidence_mtx2.shape[0]) )


    #sort_confidence = np.sort(confidence1)
    sort_confidence = np.sort(np.concatenate((confidence1, confidence2)))
    max_accuracy = 0.5
    best_precision = 0.5
    best_recall = 0.5
    for num in range(len(sort_confidence)):
        delta = sort_confidence[num]
        ratio1 = np.sum(confidence1>=delta)/confidence_mtx1.shape[0]
        ratio2 = np.sum(confidence2>=delta)/confidence_mtx2.shape[0]
        accuracy_now = 0.5*(ratio1+1-ratio2)
        if accuracy_now > max_accuracy:
            max_accuracy = accuracy_now
            best_precision = ratio1/(ratio1+ratio2)
            best_recall = ratio1
    print('maximum inference accuracy is:', max_accuracy)
    print('precision is:', best_precision)
    print('recall is:', best_recall)
    return max_accuracy


In [0]:
class Sparse(Constraint):
    '''
    We will use one variable: Mask
    After we train our model dense model,
    we will save the weights and analyze them.
    We will create a mask where 1 means the
    number is far away enough from 0 and 0
    if it is to close to 0. We will multiply
    the weights by 0(making them 0) if they
    are supposed to be masked.
    '''

    def __init__(self, mask):
        self.mask = K.cast_to_floatx(mask)

    def __call__(self,x):
        return self.mask * x

    def get_config(self):
        return {'mask': self.mask}


def create_sparsity_masks(model,sparsity):
    weights_list = model.get_weights()
    masks = []
    for weights in weights_list:
        #We can ignore biases
        if len(weights.shape) > 1:
            weights_abs = np.abs(weights)
            masks.append((weights_abs>np.percentile(weights_abs,sparsity))*1.)
    return masks

# **Dense Training**

In [5]:
epochs=50

model = Sequential([
Conv2D(32, (3,3), padding='same',  input_shape=(32,32,3)),
Activation('elu'),
BatchNormalization(),
Conv2D(32, (3,3), padding='same'),
Activation('elu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2)),
Conv2D(64, (3,3), padding='same'),
Activation('elu'),
BatchNormalization(),
Conv2D(64, (3,3), padding='same'),
Activation('elu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2)),
Conv2D(128, (3,3), padding='same'),
Activation('elu'),
BatchNormalization(),
Conv2D(128, (3,3), padding='same'),
Activation('elu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2)),
Flatten(),
Dense(10, activation='softmax')
])

adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


model.fit(x_train, y_train,
          batch_size=32,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

















Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 

In [6]:
output_train=model.predict(x_train)
output_test=model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)

model accuracy for training and test- (0.98962, 0.7977)
maximum inference accuracy is: 0.5960999999999999
precision is: 0.5290841958719206
recall is: 1.7482
Maximum Accuracy: 0.5960999999999999


#  **Sparse Training**

In [0]:
masks = create_sparsity_masks(model,10)#Closest 30% to 0

sparse_model =  Sequential([
Conv2D(32, (3,3), padding='same',  input_shape=(32,32,3), kernel_constraint=Sparse(masks[0])),
Activation('elu'),
BatchNormalization(),
Conv2D(32, (3,3), padding='same', kernel_constraint=Sparse(masks[1])),
Activation('elu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2)),
Conv2D(64, (3,3), padding='same', kernel_constraint=Sparse(masks[2])),
Activation('elu'),
BatchNormalization(),
Conv2D(64, (3,3), padding='same', kernel_constraint=Sparse(masks[3])),
Activation('elu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2)),
Conv2D(128, (3,3), padding='same', kernel_constraint=Sparse(masks[4])),
Activation('elu'),
BatchNormalization(),
Conv2D(128, (3,3), padding='same', kernel_constraint=Sparse(masks[5])),
Activation('elu'),
BatchNormalization(),
MaxPooling2D(pool_size=(2,2)),
Flatten(),
Dense(10, activation='softmax', kernel_constraint=Sparse(masks[6]))
])

adam = Adam()
sparse_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
sparse_model.summary()
#Get weights from densely trained model
sparse_model.set_weights(model.get_weights())


# Train Sparse Model
sparse_model.fit(x_train,y_train,
          batch_size=32,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))



Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 32)       

In [0]:
output_train=sparse_model.predict(x_train)
output_test=sparse_model.predict(x_test)
y_test = y_test.astype('int')
y_train = y_train.astype('int')

inference_accuracy=inference_via_confidence(output_train, output_test, y_train, y_test)
print("Maximum Accuracy:",inference_accuracy)